In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import multiprocess as mp
root.gStyle.SetOptFit(1111)
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
!rm output/dphi0_offset/?/*
sim = 0
Min_Entries = 100
if sim == 1: Min_Entries = 0

In [3]:
N_rg = 12
N_files = N_rg
N_arm = 2
N_centr = 5
pt_bins = [[5, 15],[15,20],[30,35],[35,45]]
#pt_bins = [[15, 15],[36,36]]
N_pt = len(pt_bins)
rebin = 1
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/dphi0_offset/"
file_name="dphi0_DCVTX_corr"#dphi0_DCVTX_corr dphi0_DCVTX
hist_select_3D_names = [f"dcphi0_truephi0_hist_{i}" for i in range(N_centr)]
iOptamesX=["#phi","#theta","#theta","#phi"]
iOptamesY=["#phi_{DC}-#phi_{VTX}","#theta","#phi","#theta"]

In [4]:
full_pahtes = ["../sim/output/summer25/pt/dalitz_v03.root","../VTXalignment/input/my-m_ee_Run14AuAu_135th_new_20003_1063runs.root"] # m_ee_Run14AuAu_135th_new_20003_1063runs m_ee_Run14AuAu_132nd_new_19999_900runs m_ee_Run14AuAu_128th_new_19983_1065runs
if sim: full_path=full_pahtes[sim-1]

In [5]:
hists_read = []

for ifile in range(N_files):
    infile = None
    if not sim: infile = root.TFile.Open(file_path+file_name+f"_rg{ifile}.root", "read")
    if sim:
        infile = root.TFile.Open(full_path, "read")
    hists_read_file = []
    for ihist in range(len(hist_select_3D_names)):  
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"_file{ifile}")
        hists_read_file[-1].SetDirectory(root.nullptr)
    hists_read.append(hists_read_file)
infile.Close()

In [6]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_gaus = [[-0.051,0.052],[-0.051,0.052]]
boards_pol0 = [[-0.09,0.09],[-0.09,0.09]] 

In [7]:
def misalignment_fitter(InHist, ipt, iarm, irg):
    N_gaus=1

    hist_phi_bin = InHist.ProjectionY("kek")

    ibinz0 = pt_bins[ipt][0]
    ibinz1 = pt_bins[ipt][1]

    tot_entries = dphi_porj = InHist.ProjectionX(f"dphi_tot_{ipt}_{iarm}_{irg}", 1, 120, ibinz0, ibinz1).GetEntries()

    phi_local =  array( 'd' )
    phi_local_err =  array( 'd' )

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )

    N_bins = hist_phi_bin.GetNbinsX()
    N_iter = int(N_bins/rebin)
 
    for iphi in range(N_iter):
        c1 = root.TCanvas(f"c1_{ipt}_{iphi}_{iarm}_{irg}",f"c1_{ipt}_{iphi}_{iarm}_{irg}",720,360)

        ibiny0 = iphi*rebin+1
        ibiny1 = (iphi+1)*rebin

        dphi_porj = InHist.ProjectionX(f"dphi_hist_{ipt}_{iphi}_{iarm}_{irg}", ibiny0, ibiny1, ibinz0, ibinz1)

        if dphi_porj.GetEntries() < Min_Entries:
            continue

        phi = 0.5* (hist_phi_bin.GetBinCenter(ibiny0) + hist_phi_bin.GetBinCenter(ibiny1))
        phi_err = rebin*hist_phi_bin.GetBinWidth(ibiny0)*0.5

        dphi_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{ipt}_{iphi}_{iarm}_{irg}', 'gaus',  boards_gaus[iarm][0], boards_gaus[iarm][1])
        #g_gaus2= root.TF1( f'g_gaus_2_{ilayer}_{iphi}', 'gaus',  boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol_{ipt}_{iphi}_{iarm}_{irg}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_pol0[iarm][0], boards_pol0[iarm][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_{ipt}_{iphi}_{iarm}_{irg}', 'gaus(0)+[3]+[4]*x*(abs(x)/x-1)+[5]*x*(abs(x)/x+1)',boards_tot[iarm][0], boards_tot[iarm][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_tot[iarm][1])

        dphi_porj.Draw()
        dphi_porj.Fit( g_gaus, 'QR')
        #dphi_porj.Fit( g_gaus2, 'QR+' )
        dphi_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2] = par_gaus[0], par_gaus[1], par_gaus[2]
        par_tot[3], par_tot[4], par_tot[5] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        #total.SetParLimits(1,boards_tot[iarm][0]/10,boards_tot[iarm][1]/10)
        #total.SetParLimits(2,boards_fit[iarm][1]/10,2)
        
        dphi_porj.Draw()
        dphi_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{ipt}_{iphi}_{iarm}_{irg}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])

        Format_Func(g_gaus,3,4)

        par_gaus[0], par_gaus[1], par_gaus[2] = par_tot[0], par_tot[1], par_tot[2]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[3], par_tot[4], par_tot[5]
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_pol.Draw('same')

        if abs(total.GetParameter(1))>0.45 or total.GetNDF()<20 or (total.GetParameter(2)>0.075 or total.GetParameter(2)<0.001 ): continue

        phi_local.append(phi)
        phi_local_err.append(phi_err)

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1)+total.GetParameter(2)*0.5+0*abs(total.GetParameter(2))*(dphi_porj.GetEntries()*total.GetNDF()/tot_entries)**(-0.5))
        Sigma_local_err.append(total.GetParError(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-2*total.GetParameter(2),total.GetParameter(1)+2*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-2*total.GetParameter(2),total.GetParameter(1)+2*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs(f"output/dphi0_offset/{iOption0}/fit_{ipt}_{irg}_{iarm}_{iphi}.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err,phi_local,phi_local_err, int_fg_local, int_bg_local]


In [8]:
def misalignment_layer_fitter(ifile):
    out_hists = []
    for ipt in range(N_pt):
        out_hists1 = []
        for iarm in range(N_arm):
            out_hists1.append(misalignment_fitter(hists_read[ifile][iOption0], ipt, iarm, ifile))
        out_hists.append(out_hists1)
    return out_hists

In [9]:
Ntr = N_files

pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(misalignment_layer_fitter,  [i for i in range(Ntr)])
pool.close()

In [10]:
Means, Mean_errs, Sigmas, Sigma_errs, phi_arr, phi_arr_err, Int_FG, Int_BG = [], [], [], [], [], [], [], []
for irg in range(N_rg):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, phi_arr1, phi_arr_err1, Int_FG1, Int_BG1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_pt):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, phi_arr0, phi_arr_err0, Int_FG0, Int_BG0 = [], [], [], [], [], [], [], []
        for iarm in range(N_arm):
            Means0.append(output_array[irg][ilayer][iarm][0])
            Mean_errs0.append(output_array[irg][ilayer][iarm][1])
            Sigmas0.append(output_array[irg][ilayer][iarm][2])
            Sigma_errs0.append(output_array[irg][ilayer][iarm][3])
            phi_arr0.append(output_array[irg][ilayer][iarm][4])
            phi_arr_err0.append(output_array[irg][ilayer][iarm][5])
            Int_FG0.append(output_array[irg][ilayer][iarm][6])
            Int_BG0.append(output_array[irg][ilayer][iarm][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        phi_arr1.append(phi_arr0)
        phi_arr_err1.append(phi_arr_err0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    phi_arr.append(phi_arr1)
    phi_arr_err.append(phi_arr_err1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [11]:
root.gStyle.SetOptFit(0000)

In [12]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []

for irg in range(N_rg):
    fit_functions0 = []
    for ilayer in range(N_pt):
        fit_functions1 = []
        for iarm in range(N_arm):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ilayer}_{irg}_{iarm}","[0]*sin(x) + [1]*cos(x) + [2]",-2.0,5.0))
            fit_functions1[-1].SetParameter(0,-0.016607)#, 
            fit_functions1[-1].SetParameter(1,0.005292)
            fit_functions1[-1].SetParLimits(0,-0.05,0.05)
            fit_functions1[-1].FixParameter(0,0)
            fit_functions1[-1].FixParameter(1,0)
            
            #fit_functions1[-1].SetParameter(0,0.001)
            #fit_functions1[-1].SetParameter(2,0.001)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [13]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions_kek = []

for irg in range(N_rg):
    fit_functions0 = []
    for ilayer in range(N_pt):
        fit_functions1 = []
        for iarm in range(N_arm):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ilayer}_{irg}_{iarm}","[0]*sin(x) + [1]*cos(x) + [2]",-2.0,5.0))
            fit_functions1[-1].SetParameter(0,-0.016607)#, 
            fit_functions1[-1].SetParameter(1,0.005292)
            fit_functions1[-1].SetParLimits(0,-0.05,0.05)
            fit_functions1[-1].FixParameter(0,0)
            fit_functions1[-1].FixParameter(1,0)
            
            #fit_functions1[-1].SetParameter(0,0.001)
            #fit_functions1[-1].SetParameter(2,0.001)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions_kek.append(fit_functions0)

In [30]:
irg = 0
h2=[]
c2 = root.TCanvas(f"c2_{iarm}",f"c2_{iarm}",1000,400*int(max(2/2,1)))
c2.Divide(2,1)
legend_names = ["neg high pt", "neg low pt", "pos low pt", "pos high pt"]
g_mean = []
min0, max0 = -0.04, 0.079
x0, x1 = 1.5, 4
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for iarm in range(2):
    if iarm ==1 :
        if iOption0==0: x0, x1 = -1, 1.55
    g_mean_rg = []
    c2.cd(iarm+1)

    h2.append(Format_Pad_old( x0, x1, min0+0.008, max0, f"{iOptamesX[iOption0]}"+"_{VTX} (rad)", f"#LT{iOptamesY[iOption0]}#GT", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))

    h2[-1].GetXaxis().SetTitleSize(0.06)
    h2[-1].GetXaxis().SetLabelSize(0.05)
    h2[-1].GetXaxis().SetTitleOffset(1.2)
    h2[-1].GetYaxis().SetTitleSize(0.06)
    h2[-1].GetYaxis().SetLabelSize(0.05)
    h2[-1].GetYaxis().SetTitleOffset(1.2)
    h2[-1].GetYaxis().SetNdivisions(808)
    h2[-1].GetYaxis().CenterTitle()
    h2[-1].GetXaxis().CenterTitle()
    h2[-1].Draw()
    root.gPad.SetBottomMargin(0.15)
    root.gPad.SetLeftMargin(0.15)
    root.gPad.SetTopMargin(0.01)
    root.gPad.SetRightMargin(0.01) 
    legends.append(root.TLegend(0.2,0.7,0.95,0.99))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    for ilayer in range(0,N_pt,1):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Means[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Mean_errs[irg][ilayer][iarm])
        Format_Graph(g_mean_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean_layer.Fit(fit_functions_kek[irg][ilayer][iarm],"RQ","",x0+0.4, x1-0.2)
        g_mean_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean_layer,legend_names[ilayer],"p")
        else: legends[-1].AddEntry(g_mean_layer,legend_names[ilayer],"p")
        g_mean_rg.append(g_mean_layer)
    legends[-1].Draw()
    g_mean.append(g_mean_rg)
c2.Draw()
c2.SaveAs(f"output/dphi0_offset/fit_arm_rg0.png")

In [15]:
# ...existing code...
import copy

Mean_before = copy.deepcopy(Means)  # independent copy
for iarm in range(N_arm):
    for ipt in range(N_pt):
        for iphi in range(len(Mean_before[0][ipt][iarm])):
            if phi_arr[0][ipt][iarm][iphi]>1.5: Mean_before[0][ipt][iarm][iphi]-=0.01
            if phi_arr[0][ipt][iarm][iphi]<1.5: Mean_before[0][ipt][iarm][iphi]+=0.02

In [29]:
irg = 0
h2=[]
c2 = root.TCanvas(f"c2_{iarm}",f"c2_{iarm}",1000,400*int(max(2/2,1)))
c2.Divide(2,1)
legend_names = ["neg high pt", "neg low pt", "pos low pt", "pos high pt"]
g_mean = []
min0, max0 = -0.04, 0.079
x0, x1 = 1.5, 4
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for iarm in range(2):
    if iarm ==1 :
        if iOption0==0: x0, x1 = -1, 1.55
    g_mean_rg = []
    c2.cd(iarm+1)

    h2.append(Format_Pad_old( x0, x1, min0+0.008, max0, f"{iOptamesX[iOption0]}"+"_{VTX} (rad)", f"#LT{iOptamesY[iOption0]}#GT", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))

    h2[-1].GetXaxis().SetTitleSize(0.06)
    h2[-1].GetXaxis().SetLabelSize(0.05)
    h2[-1].GetXaxis().SetTitleOffset(1.2)
    h2[-1].GetYaxis().SetTitleSize(0.06)
    h2[-1].GetYaxis().SetLabelSize(0.05)
    h2[-1].GetYaxis().SetTitleOffset(1.2)
    h2[-1].GetYaxis().SetNdivisions(808)
    h2[-1].GetYaxis().CenterTitle()
    h2[-1].GetXaxis().CenterTitle()
    h2[-1].Draw()
    root.gPad.SetBottomMargin(0.15)
    root.gPad.SetLeftMargin(0.15)
    root.gPad.SetTopMargin(0.01)
    root.gPad.SetRightMargin(0.01) 
    legends.append(root.TLegend(0.2,0.7,0.95,0.99))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    for ilayer in range(0,N_pt,1):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Mean_before[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Mean_errs[irg][ilayer][iarm])
        Format_Graph(g_mean_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean_layer.Fit(fit_functions_kek[irg+1][ilayer][iarm],"RQ","",x0+0.2, x1-0.2)
        g_mean_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean_layer,legend_names[ilayer],"p")
        else: legends[-1].AddEntry(g_mean_layer,legend_names[ilayer],"p")
        g_mean_rg.append(g_mean_layer)
    legends[-1].Draw()
    g_mean.append(g_mean_rg)
c2.Draw()
c2.SaveAs(f"output/dphi0_offset/fit_nocorr_arm_rg0.png")

In [27]:
iarm = 0
h2=[]
c2 = root.TCanvas(f"c2_{iarm}",f"c2_{iarm}",1000,400*int(max(N_rg/2,1)))
c2.Divide(2,int(max(N_rg/2,1)))
g_mean = []
min0, max0 = -0.099, 0.099
x0, x1 = 1.5, 4
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for irg in range(N_rg):
    if False and irg ==1 :
        iarm = 1
        if iOption0==0: x0, x1 = -1, 1.5
    g_mean_rg = []
    c2.cd(irg+1)
    h2.append(Format_Pad_old( x0, x1, min0+0.008, max0, f"{iOptamesX[iOption0]}"+"_{VTX} (rad)", f"#LT{iOptamesY[iOption0]}#GT", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h2[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    for ilayer in range(0,N_pt,1):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Means[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Mean_errs[irg][ilayer][iarm])
        Format_Graph(g_mean_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean_layer.Fit(fit_functions[irg][ilayer][iarm],"RQ","",2,3.9)
        g_mean_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean_layer,f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_mean_layer,f"neg layer {ilayer}","p")
        g_mean_rg.append(g_mean_layer)
    legends[-1].Draw()
    g_mean.append(g_mean_rg)
c2.Draw()
c2.SaveAs(f"output/dphi0_offset/fit_{iarm}.png")

In [26]:
iarm = 1
h2=[]
c3 = root.TCanvas(f"c3_{iarm}",f"c3_{iarm}",1000,400*int(max(N_rg/2,1)))
c3.Divide(2,int(max(N_rg/2,1)))
g_mean = []
min0, max0 = -0.035, 0.035
x0, x1 = -1, 1.5
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for irg in range(N_rg):
    g_mean_rg = []
    c3.cd(irg+1)
    h2.append(Format_Pad_old( x0, x1, min0+0.008, max0, f"{iOptamesX[iOption0]}0 (rad)", f"#LT{iOptamesY[iOption0]}#GT", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h2[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    for ilayer in range(0,N_pt,1):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Means[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Mean_errs[irg][ilayer][iarm])
        Format_Graph(g_mean_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean_layer.Fit(fit_functions[irg][ilayer][iarm],"QR","",-1,1.2)
        g_mean_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean_layer,f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_mean_layer,f"neg layer {ilayer}","p")
        g_mean_rg.append(g_mean_layer)
    legends[-1].Draw()
    g_mean.append(g_mean_rg)
c3.Draw()
#c2.SaveAs(f"output/dphi0_offset/fit_{iarm}.pdf")

In [19]:
if True:
    ilayer = 0
    print(f"const float phi0_DC_VTX_offset_params[{N_arm}][2] = "+"{")
    for iarm in range(N_arm):
        iirg = irg
        #if iOption0 == 1 and irg>4 and iarm == 0:
        #    if irg < 8: iirg = 4
        #    elif irg < 10: iirg = 11
        par0 = 0.5*(fit_functions[iirg][ilayer+0][iarm].GetParameter(0)+fit_functions[iirg][ilayer+1][iarm].GetParameter(0))
        par1 = 0.5*(fit_functions[iirg][ilayer+0][iarm].GetParameter(1)+fit_functions[iirg][ilayer+1][iarm].GetParameter(1))
        print("    {"+f"{round(par0,6)}, {round(par1,6)}"+"},")
    print("};")

const float phi0_DC_VTX_offset_params[2][2] = {
    {0.0, 0.0},
    {0.0, 0.0},
};


In [20]:
if True:
    ilayer = 0
    print(f"const float "+iOptamesX[iOption0]+f"_offset_params[{N_rg}][{N_arm}][2][3] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            iirg = irg
            #if iOption0 == 1 and irg>4 and iarm == 0:
            #    if irg < 8: iirg = 4
            #    elif irg < 10: iirg = 11
            print("        {")     
            print("            {"+f"{round(fit_functions[iirg][ilayer+0][iarm].GetParameter(0),6)}, {round(fit_functions[iirg][ilayer+0][iarm].GetParameter(1),6)}, {round(fit_functions[iirg][ilayer+0][iarm].GetParameter(2),6)}"+"},")
            print("            {"+f"{round(fit_functions[iirg][ilayer+1][iarm].GetParameter(0),6)}, {round(fit_functions[iirg][ilayer+1][iarm].GetParameter(1),6)}, {round(fit_functions[iirg][ilayer+1][iarm].GetParameter(2),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")



const float #phi_offset_params[12][2][2][3] = {
    {
        {
            {0.0, 0.0, 0.000993},
            {0.0, 0.0, 0.001309},
        },
        {
            {0.0, 0.0, 0.000974},
            {0.0, 0.0, 0.000936},
        },
    },
    {
        {
            {0.0, 0.0, 0.000792},
            {0.0, 0.0, 0.000327},
        },
        {
            {0.0, 0.0, 0.00083},
            {0.0, 0.0, 0.0009},
        },
    },
    {
        {
            {0.0, 0.0, 0.000904},
            {0.0, 0.0, 0.001158},
        },
        {
            {0.0, 0.0, 0.000968},
            {0.0, 0.0, 0.000936},
        },
    },
    {
        {
            {0.0, 0.0, 0.000875},
            {0.0, 0.0, 0.000298},
        },
        {
            {0.0, 0.0, 0.000905},
            {0.0, 0.0, 0.000877},
        },
    },
    {
        {
            {0.0, 0.0, 0.00067},
            {0.0, 0.0, 4.1e-05},
        },
        {
            {0.0, 0.0, 0.000952},
            {0.0, 0.0, 0.00088},
        },
    },
   

In [21]:
if True:
    ilayer = 0
    print(f"const float "+iOptamesX[iOption0]+f"_offset_params[{N_rg}][{N_arm}] = "+"{")
    for irg in range(N_rg):
        print("{")
        for iarm in range(N_arm):
            iirg = irg
            #if iOption0 == 1 and irg>4 and iarm == 0:
            #    if irg < 8: iirg = 4
            #    elif irg < 10: iirg = 11
            print("    {"+f"{(fit_functions[iirg][ilayer+0][iarm].GetParameter(2)+fit_functions[iirg][ilayer+1][iarm].GetParameter(2))/2.:<6.6f}"+"},")
        print("},")   
    print("};")


const float #phi_offset_params[12][2] = {
{
    {0.001151},
    {0.000955},
},
{
    {0.000559},
    {0.000865},
},
{
    {0.001031},
    {0.000952},
},
{
    {0.000586},
    {0.000891},
},
{
    {0.000356},
    {0.000916},
},
{
    {0.000366},
    {0.000864},
},
{
    {0.000373},
    {0.000959},
},
{
    {0.000263},
    {0.001005},
},
{
    {0.001036},
    {0.001190},
},
{
    {0.000461},
    {0.000979},
},
{
    {0.000250},
    {0.001032},
},
{
    {0.000158},
    {0.001024},
},
};


In [22]:
if False:
    print(f"const float "+iOptames[iOption0]+f"_offset_param0[{N_rg}][{N_arm}][2][4] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[0][irg][iarm].GetParameter(0),6)}, {round(fit_functions[1][irg][iarm].GetParameter(0),6)}, {round(fit_functions[2][irg][iarm].GetParameter(0),6)}, {round(fit_functions[3][irg][iarm].GetParameter(0),6)}"+"},")
            print("            {"+f"{round(fit_functions[4][irg][iarm].GetParameter(0),6)}, {round(fit_functions[5][irg][iarm].GetParameter(0),6)}, {round(fit_functions[6][irg][iarm].GetParameter(0),6)}, {round(fit_functions[7][irg][iarm].GetParameter(0),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")
    print(f"const float "+iOptames[iOption0]+f"_offset_param1[{N_rg}][{N_arm}][2][4] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[0][irg][iarm].GetParameter(1),6)}, {round(fit_functions[1][irg][iarm].GetParameter(1),6)}, {round(fit_functions[2][irg][iarm].GetParameter(1),6)}, {round(fit_functions[3][irg][iarm].GetParameter(1),6)}"+"},")
            print("            {"+f"{round(fit_functions[4][irg][iarm].GetParameter(1),6)}, {round(fit_functions[5][irg][iarm].GetParameter(1),6)}, {round(fit_functions[6][irg][iarm].GetParameter(1),6)}, {round(fit_functions[7][irg][iarm].GetParameter(1),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")
    print(f"const float "+iOptames[iOption0]+f"_offset_param2[{N_rg}][{N_arm}][2][4] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[0][irg][iarm].GetParameter(2),6)}, {round(fit_functions[1][irg][iarm].GetParameter(2),6)}, {round(fit_functions[2][irg][iarm].GetParameter(2),6)}, {round(fit_functions[3][irg][iarm].GetParameter(2),6)}"+"},")
            print("            {"+f"{round(fit_functions[4][irg][iarm].GetParameter(2),6)}, {round(fit_functions[5][irg][iarm].GetParameter(2),6)}, {round(fit_functions[6][irg][iarm].GetParameter(2),6)}, {round(fit_functions[7][irg][iarm].GetParameter(2),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")     
     


In [23]:
if False:
    ilayer, ilayer1 = 1, 5
    if iOption0: print(f"const float the_offset_params[{N_rg}][{N_arm}][3] = "+"{")
    else: print(f"const float phi_offset_params[{N_rg}][{N_arm}][3] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {"+f"{round(fit_functions[ilayer][irg][iarm].GetParameter(0)+fit_functions[ilayer1][irg][iarm].GetParameter(0),6)}, \
    {round(fit_functions[ilayer][irg][iarm].GetParameter(1)+fit_functions[ilayer1][irg][iarm].GetParameter(1),6)}, \
    {round(fit_functions[ilayer][irg][iarm].GetParameter(2)+fit_functions[ilayer1][irg][iarm].GetParameter(2),6)}"+"},")
        print("    },")   
    print("};")        

In [24]:
iarm = 0
h4=[]
c4 = root.TCanvas(f"c4_{iarm}",f"c4_{iarm}",1000,400*int(N_rg/2))
c4.Divide(2,int(N_rg/2))
min0, max0 = -0.015, 0.015
x0, x1 = 1.5, 4
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
g_sigma = []
for irg in range(N_rg):
    c4.cd(irg+1)
    h4.append(Format_Pad_old( x0, x1, 0, max0, f"{iOptamesX[iOption0]}0 (rad)", f"sigma d{iOptamesY[iOption0]}", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h4[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    g_sigma_rg = []
    for ilayer in range(0,N_pt):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_sigma_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Sigmas[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Sigma_errs[irg][ilayer][iarm])
        Format_Graph(g_sigma_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_sigma_layer.Fit("pol0","Q","")
        g_sigma_layer.GetFunction("pol0").SetLineColor(colors[ilayer])
        g_sigma_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_sigma_layer,f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_sigma_layer,f"neg layer {ilayer}","p")
        g_sigma_rg.append(g_sigma_layer)
    g_sigma.append(g_sigma_rg)
    legends[-1].Draw()
c4.Draw()
c4.SaveAs(f"output/misalignment/sigma_{iarm}_{iOption0}.png")

In [25]:
iarm = 1
h4=[]
c5 = root.TCanvas(f"c5_{iarm}",f"c5_{iarm}",1000,400*int(N_rg/2))
c5.Divide(2,int(N_rg/2))
min0, max0 = -0.015, 0.015
x0, x1 =-1, 1.5
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
g_sigma = []
for irg in range(N_rg):
    c5.cd(irg+1)
    h4.append(Format_Pad_old( x0, x1, 0, max0, f"{iOptamesX[iOption0]}0 (rad)", f"sigma d{iOptamesY[iOption0]}", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h4[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    g_sigma_rg = []
    for ilayer in range(0,N_pt):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_sigma_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Sigmas[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Sigma_errs[irg][ilayer][iarm])
        Format_Graph(g_sigma_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_sigma_layer.Fit("pol0","Q","")
        g_sigma_layer.GetFunction("pol0").SetLineColor(colors[ilayer])
        g_sigma_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_sigma_layer,f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_sigma_layer,f"neg layer {ilayer}","p")
        g_sigma_rg.append(g_sigma_layer)
    g_sigma.append(g_sigma_rg)
    legends[-1].Draw()
c5.Draw()
c5.SaveAs(f"output/misalignment/sigma_{iarm}_{iOption0}.png")